# Analiza opinii o kupionych produktach

## Zacznijmy od importu potrzebnych bibiotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

## Wczytajmy plik zawierający opinie o kupionych produktach dziecięcych

In [ ]:
#products = pd.read_csv('https://raw.githubusercontent.com/shiv-source/Product-sentiment-analysis/master/amazon_baby.csv')
products = pd.read_csv('dane/amazon_baby.csv')

## Przyjrzyjmy się tym danym

In [ ]:
products.head()

## Które produkty otrzymały najwięcej opinii?

In [ ]:
products['name'].value_counts()

##### Dla zainteresowanych: https://www.smyk.com/p/zyrafka-sophie-so-pure-gryzak-i6022444?gclid=EAIaIQobChMIzf2C99vF-AIV2eJ3Ch17cAlUEAQYASABEgKBQPD_BwE

## Przyjrzyjmy się opiniom na temat wybranego produktu

### Liczba opinii

W poniższej linijce możesz eksperymentować z różnymi produktami

In [ ]:
my_prod_reviews = products.loc[products['name'] == 'Vulli Sophie the Giraffe Teether']
my_prod_reviews.size

### Jaki jest rozkład ocen wybranego produktu?

In [ ]:
my_prod_reviews.rating.value_counts()

In [ ]:
my_prod_reviews.rating.mean()

## Zbudujmy model klasyfikujący opinię jako pozytywną lub negatywną

In [ ]:
#vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
vectorizer = CountVectorizer(token_pattern=r'\b[a-zA-z]{3,}\b')
counts = vectorizer.fit_transform(products.review.values.astype('U'))
#print(vectorizer.vocabulary_)

### (liczba opinii, liczba słów w worku)

In [ ]:
counts.shape

### Kryteria pozytywnej i negatywnej oceny

#### Zignorujmy oceny neutralne zawierające 3 gwiazdki

In [ ]:
products = products[products['rating']!= 3]

#### Pozytywna opinia = 4 lub 5 gwiazdek

In [ ]:
products['sentiment'] = products['rating'].apply(lambda rating: 1 if rating >= 4 else -1)
products.sort_values(by='sentiment', ascending=False)

### Czas na wytrenowanie naszego klasyfikatora opinii

#### Dzielimy opinie na zbiory treningowy i testowy

In [ ]:
train, test = train_test_split(products, test_size=0.2, random_state=0)

#### Zbieramy do worka słowa zawarte w zbiorze treningowym

In [ ]:
train_matrix = vectorizer.fit_transform(train.review.values.astype('U'))

#### Przekształcamy opinie w zbiorze testowym do podobnej postaci

In [ ]:
test_matrix = vectorizer.transform(test.review.values.astype('U'))

#### Wyciągamy ze zbiórów oceny przypisane do recenzji

In [ ]:
y_train = train.sentiment
y_test = test.sentiment

#### Przystępujemy do uczenia modelu

In [ ]:
sentiment_model = LogisticRegression().fit(train_matrix, y_train)

### Sprawdźmy jak spisuje się nasz klasyfikator

In [ ]:
test['predicted_sentiment'] = sentiment_model.predict(test_matrix)
test.head()

#### Tablica pomyłek

In [ ]:
predictions = test.predicted_sentiment
confusion_matrix(predictions, y_test)

#### Precyzja, czułość etc.

In [ ]:
print(classification_report(predictions,y_test))

#### Wydzielamy opinie, które zostały błędnie sklasyfikowane

In [ ]:
false_pred = test[test.sentiment != test.predicted_sentiment]
fp = false_pred[false_pred.sentiment == -1]
fn = false_pred[false_pred.sentiment == 1]

Opinia negatywna, która została sklasyfikowana jako pozytywna

In [ ]:
fp.review.iloc[0]

Opinia pozytywna, która została sklasyfikowana jako negatywna

In [ ]:
fn.review.iloc[0]

#### Dla wybranego produktu postarajmy się ustalić przyczyny błędnej klasyfikacji

In [ ]:
my_prod_test = test.loc[products['name'] == 'Vulli Sophie the Giraffe Teether']
false_pred = my_prod_test[my_prod_test.sentiment != my_prod_test.predicted_sentiment]
fp = false_pred[false_pred.sentiment == -1]
fn = false_pred[false_pred.sentiment == 1]

Opinia negatywna, która została sklasyfikowana jako pozytywna

In [ ]:
fp.review.iloc[0]

Opinia pozytywna, która została sklasyfikowana jako negatywna

In [ ]:
fn.review.iloc[1]